In [355]:
from openstack import connection, compute, database
import copy

In [393]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn

In [187]:
class OSmap(object):
    def __init__ (self, connection='', OS=True):
        'Инит функции'
        if OS == True:
            self.__conn = connection
            self.__sl = [i for i in self.__conn.compute.servers(all_tenants=True)]
            self.__fd = {}
            for i in self.__conn.compute.flavors():
                self.__fd[i.id] = i.to_dict()
            self.fd = copy.deepcopy(self.__fd)
            
    def get_names (self):
        'Функция возвращает список имен ВМ'
        a = [i.name for i in self.__sl]
        return a
    def get_vm_by_name(self, vmn=''):
        'Функция возвращает объект ВМ на основании имени'
        for i in self.__sl:
            if i.name == vmn:
                return i
    def get_vm_dict(self):
        'Возвращает словарь с описанием ВМ из OS + cpu & ram'
        d = {}
        for i in self.__sl:
            d[i.name] = self.get_vm_by_name(vmn=i.name).to_dict()
            print(i.name,i.flavor['id'])
            d[i.name]['cpu'] = self.__fd[i.flavor['id']]['vcpus']
            d[i.name]['ram'] = self.__fd[i.flavor['id']]['ram']
            
        return d
        

In [188]:
%%time
x = OSmap(connection_to_cluster('admin'))

CPU times: user 545 ms, sys: 37.7 ms, total: 583 ms
Wall time: 17.1 s


In [189]:
x.fd['5de6871b-ba51-4e4e-be7f-1d44007f1dcc']

{'disk': 72,
 'ephemeral': 0,
 'id': '5de6871b-ba51-4e4e-be7f-1d44007f1dcc',
 'is_disabled': False,
 'is_public': True,
 'links': [{'href': 'http://controller:8774/v2.1/b83a5fb65edc41a1a745f06aa093d416/flavors/5de6871b-ba51-4e4e-be7f-1d44007f1dcc',
   'rel': 'self'},
  {'href': 'http://controller:8774/b83a5fb65edc41a1a745f06aa093d416/flavors/5de6871b-ba51-4e4e-be7f-1d44007f1dcc',
   'rel': 'bookmark'}],
 'location': None,
 'name': '8x32768x72',
 'ram': 32768,
 'rxtx_factor': 1.0,
 'swap': '',
 'vcpus': 8}

In [190]:
%%time
fd = x.get_vm_dict()

make_FreeBSD_great_again 8724e0a0-e209-4025-a272-f04fab547cfa
is_efem_vm 76520a4b-7682-4daa-9997-cf8323f3389c
is_efem 3905118a-ca74-43be-9e54-4a1370be6042
9.4.gerc_new 9b732cc9-e5b7-41b2-87a0-3f70bb59b91c
ppoz-popd-process-05 8724e0a0-e209-4025-a272-f04fab547cfa
egron-elasticsearch-03 8200c165-9f19-4cdb-bae7-8b0167f06d28
migr-node-079 5de6871b-ba51-4e4e-be7f-1d44007f1dcc
tnx-mon 3905118a-ca74-43be-9e54-4a1370be6042
agent-pvd3 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-worker-022 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-039 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-038 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-037 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-036 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-035 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-034 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-033 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-032 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-031 8724e0a0-e209-4025-a272-f04fab547cfa
pkurp-app-030 8724e0

KeyError: 'f251edf6-93e0-419a-b0d4-0c2091be4f98'

In [191]:
fd['migr-analitycs-01']

{'access_ipv4': '',
 'access_ipv6': '',
 'addresses': {'vlan_20': [{'OS-EXT-IPS-MAC:mac_addr': 'fa:16:3e:11:f3:86',
    'OS-EXT-IPS:type': 'fixed',
    'addr': '10.135.10.181',
    'version': 4}]},
 'admin_password': None,
 'attached_volumes': [{'id': 'ba20f28b-d538-4fe1-93c1-062dcea93d5f'},
  {'id': '02e7b15e-efba-46b0-aebc-4e340a134af2'}],
 'availability_zone': 'nova',
 'block_device_mapping': None,
 'created_at': '2017-07-24T20:14:30Z',
 'disk_config': 'MANUAL',
 'flavor': {'id': 'f251edf6-93e0-419a-b0d4-0c2091be4f98',
  'links': [{'href': 'http://controller:8774/b83a5fb65edc41a1a745f06aa093d416/flavors/f251edf6-93e0-419a-b0d4-0c2091be4f98',
    'rel': 'bookmark'}]},
 'flavor_id': None,
 'has_config_drive': '',
 'host_id': '680e087694930dc0a843f0c41fe524f140767c6ea74605478a34fb8d',
 'hypervisor_hostname': 'cn-e0ba.egron.tnx',
 'id': '4a73e80d-c37d-44fd-89e7-4eefcf1fada2',
 'image': {},
 'image_id': None,
 'instance_name': 'instance-0000302d',
 'key_name': None,
 'launched_at': '2017

In [33]:
vm = x.get_names()[0]

In [34]:
dir(x.get_vm_by_name(vmn=vm))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_action',
 '_alternate_id',
 '_body',
 '_body_mapping',
 '_collect_attrs',
 '_consume_attrs',
 '_filter_component',
 '_get_id',
 '_get_mapping',
 '_get_one_match',
 '_header',
 '_header_mapping',
 '_metadata',
 '_prepare_request',
 '_query_mapping',
 '_translate_response',
 '_update',
 '_uri',
 '_uri_mapping',
 'access_ipv4',
 'access_ipv6',
 'add_fixed_ip',
 'add_floating_ip',
 'add_security_group',
 'addresses',
 'admin_password',
 'allow_create',
 'allow_delete',
 'allow_get',
 'allow_head',
 'allow_list',
 'allow_update',
 'attached_volumes',
 'availability_zone',
 'backup',
 'base_path',
 'block_device_mapping',
 'c

In [126]:
s=x.get_vm_by_name(vmn=vm)
f=[i for i in c1.compute.flavors(details=True)]
s.flavor['id'],f[12]

('8724e0a0-e209-4025-a272-f04fab547cfa',
 openstack.compute.v2.flavor.FlavorDetail(name=m1.large, links=[{'href': 'http://controller:8774/v2.1/b83a5fb65edc41a1a745f06aa093d416/flavors/4', 'rel': 'self'}, {'href': 'http://controller:8774/b83a5fb65edc41a1a745f06aa093d416/flavors/4', 'rel': 'bookmark'}], ram=8192, OS-FLV-DISABLED:disabled=False, vcpus=4, swap=, os-flavor-access:is_public=True, rxtx_factor=1.0, OS-FLV-EXT-DATA:ephemeral=0, disk=80, id=4))

In [40]:
vmd = {}
for i in x.get_names():
    vmd[i]=x.get_vm_by_name(vmn=i).to_dict()

In [98]:
vmd[vm]

{'access_ipv4': '',
 'access_ipv6': '',
 'addresses': {'vlan_19': [{'OS-EXT-IPS-MAC:mac_addr': 'fa:16:3e:c4:eb:50',
    'OS-EXT-IPS:type': 'fixed',
    'addr': '10.135.7.61',
    'version': 4}]},
 'admin_password': None,
 'attached_volumes': [{'id': '887fd27a-e899-4e23-9579-eca1d25f4b3c'},
  {'id': '4f0e1f50-10bf-462d-a5a9-281a171fe628'}],
 'availability_zone': 'nova',
 'block_device_mapping': None,
 'created_at': '2017-09-05T13:48:44Z',
 'disk_config': 'AUTO',
 'flavor': {'id': '8724e0a0-e209-4025-a272-f04fab547cfa',
  'links': [{'href': 'http://controller:8774/b83a5fb65edc41a1a745f06aa093d416/flavors/8724e0a0-e209-4025-a272-f04fab547cfa',
    'rel': 'bookmark'}]},
 'flavor_id': None,
 'has_config_drive': '',
 'host_id': 'c083ddfdb234c4c4d6a22c423a9d41f0a661cc5722e3d17a40735558',
 'hypervisor_hostname': 'cn-db3e.egron.tnx',
 'id': '19aa6baa-0d23-4754-aef3-c299aa86ae75',
 'image': {'id': 'bbca591a-593c-4ad0-ae3b-8797ee0eeb29',
  'links': [{'href': 'http://controller:8774/b83a5fb65edc41

In [395]:
c1 = connection_to_cluster('admin')

In [279]:
%%time
f1 = [i for i in c1.compute.flavors()]
s1 = [i for i in c1.compute.servers(all_tenants=True)]

CPU times: user 437 ms, sys: 123 ms, total: 560 ms
Wall time: 17.1 s


In [311]:
dir(c1.compute.flavors), c1.compute.flavors.__code__.co_consts

(['__call__',
  '__class__',
  '__delattr__',
  '__dir__',
  '__doc__',
  '__eq__',
  '__format__',
  '__func__',
  '__ge__',
  '__get__',
  '__getattribute__',
  '__gt__',
  '__hash__',
  '__init__',
  '__init_subclass__',
  '__le__',
  '__lt__',
  '__ne__',
  '__new__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__self__',
  '__setattr__',
  '__sizeof__',
  '__str__',
  '__subclasshook__'],
 ('Return a generator of flavors\n\n        :param bool details: When ``True``, returns\n            :class:`~openstack.compute.v2.flavor.FlavorDetail` objects,\n            otherwise :class:`~openstack.compute.v2.flavor.Flavor`.\n            *Default: ``True``*\n        :param kwargs \\*\\*query: Optional query parameters to be sent to limit\n                                 the flavors being returned.\n\n        :returns: A generator of flavor objects\n        ',
  'paginated',
  True))

In [146]:
fl[0].id, s1[0].id

('1', '19aa6baa-0d23-4754-aef3-c299aa86ae75')

In [370]:

f2 = [i for i in c1.compute.flavors()]
a = [i.id for i in f2]
len(a)

In [371]:

f2 = [i for i in c1.compute.flavors(limit=0)]
a = [i.id for i in f2]
len(a)

50

In [406]:

f2 = [i for i in c1.compute.flavors()]
a = [i.id for i in f2]
len(a)

49

['edff431d-5808-4c01-96cb-da7de57a1fb9',
 'e6f555e2-0819-4171-96b1-6d753d51e029',
 'de35d280-5bb0-4767-ae4e-47174c00e82c',
 'dcf4f459-e849-470f-998f-0fa7dce0f8c0',
 'd38c13cc-b884-4de6-bae6-80592f06e041',
 'd115d3e0-4337-4c5f-9fdc-5246fda2deee',
 'd03d1f24-905e-409d-938b-6bd25bd69cd6',
 'cf5637fd-0565-491f-b096-3ae34446a028',
 'c0e57773-70df-48f5-b42a-9644f317e260',
 'bf13e22b-b049-401f-be7a-69e920c82573',
 'b9062ce8-fb20-450f-b214-35af445c1ded',
 'b50c24ce-4d9a-4997-9cd1-eb0703936d18',
 'b3e7c454-2034-4c95-8a31-e4b323e8c723',
 'afea8f0c-7ff5-44da-abb9-ad1ad4c2b754',
 'a77a6208-6163-4744-981a-19e6c0071787',
 '9e4eec8e-3645-4da3-b109-fab04452ccfd',
 '9b732cc9-e5b7-41b2-87a0-3f70bb59b91c',
 '99a9a5c9-83b1-4e76-b3dd-d01c14a47512',
 '9732791a-8621-40fb-a182-4f34158e0198',
 '93a4ea13-8a52-43b8-a56b-cb168ed60a91',
 '932803cf-780a-436f-9f76-16b9224dbfef',
 '8819812a-aff9-4ee9-ad57-719d5332792e',
 '87253188-b8c0-489c-a6e0-14c6e013cf14',
 '8724e0a0-e209-4025-a272-f04fab547cfa',
 '8200c165-9f19-

In [402]:

for m in ['1','2','3']:
    for l in [10,0,-1,'my_string']:
        try:
            f2 = [i for i in c1.compute.flavors(limit=l, marker=m)]
            a = [i.id for i in f2]
            print('limit:',l,'len of list',len(f2))
        except Exception as e: print('limit:',l, 'exception:',e)

limit: 10 len of list 49
limit: 0 len of list 0
limit: -1 exception: HttpException: Bad Request (HTTP 400), Invalid input received: limit must be >= 0
limit: my_string exception: HttpException: Bad Request (HTTP 400), Invalid input received: limit must be an integer
limit: 10 len of list 44
limit: 0 len of list 0
limit: -1 exception: HttpException: Bad Request (HTTP 400), Invalid input received: limit must be >= 0
limit: my_string exception: HttpException: Bad Request (HTTP 400), Invalid input received: limit must be an integer
limit: 10 len of list 42
limit: 0 len of list 0
limit: -1 exception: HttpException: Bad Request (HTTP 400), Invalid input received: limit must be >= 0
limit: my_string exception: HttpException: Bad Request (HTTP 400), Invalid input received: limit must be an integer


In [377]:
#c1.compute.get_flavor(flavor='f251edf6-93e0-419a-b0d4-0c2091be4f98')

In [268]:
y = [i.id for i in b]

In [269]:
len(y)

50

In [270]:
y

['1',
 '12918a31-586d-4874-9bbc-1e1a7a749d0c',
 '1568d5bc-8be0-4de4-b4bd-cae32b6d3c96',
 '1b78f715-a0e7-4a82-8eae-ae68531b3d6a',
 '1bdca3df-8ac0-4aca-8ae7-95179400346b',
 '2',
 '2e773749-62da-4776-84a2-337a0fa3562b',
 '3',
 '36fe0782-15f6-4f2e-a2a1-a8555f3141bd',
 '3905118a-ca74-43be-9e54-4a1370be6042',
 '3af82d8c-a6aa-4f74-9459-53b062061750',
 '3afca19f-8eb8-4917-a59a-e86dd80dd123',
 '4',
 '443050b2-680a-48da-ac63-e13f5b0733af',
 '45270476-c2f7-433e-b79d-9e4538524e20',
 '5',
 '50e1bc15-bb98-4854-abbc-6058159b6a8c',
 '592ec825-c5e6-417c-9be4-6e563ea33c3e',
 '5de6871b-ba51-4e4e-be7f-1d44007f1dcc',
 '5e5c289a-9346-438f-be8b-aae22296bbeb',
 '661e13d9-ca50-4199-a802-abf6388d8495',
 '6a33fab8-fbea-472c-970c-24365c3b4b5f',
 '75102078-bf08-48e1-9d10-f7cd9a5089dc',
 '76520a4b-7682-4daa-9997-cf8323f3389c',
 '76ad63c2-129a-4dec-bcc2-db88cdcf2610',
 '8200c165-9f19-4cdb-bae7-8b0167f06d28',
 '8724e0a0-e209-4025-a272-f04fab547cfa',
 '87253188-b8c0-489c-a6e0-14c6e013cf14',
 '8819812a-aff9-4ee9-ad57-7